In [ ]:
import requests
import os
from dotenv import load_dotenv

load_dotenv(override=True)

gemini_api_key = os.getenv("GEMINI_API_KEY")
if gemini_api_key is None:
    raise ValueError("GEMINI_API_KEY not found in .env file")

url = "https://generativelanguage.googleapis.com/v1beta/models/gemini-3-flash-preview:generateContent"

headers = {
    "Content-Type": "application/json",
    "x-goog-api-key": gemini_api_key
}

def get_ai_response(user_message, max_tokens=1500, temperature=0.7):
    """Simple function to get AI response from Gemini API"""
    payload = {
        "contents": [
            {
                "parts": [
                    {
                        "text": user_message
                    }
                ]
            }
        ],
        "generationConfig": {
            "temperature": temperature,
            "maxOutputTokens": max_tokens
        }
    }
    
    response = requests.post(url, json=payload, headers=headers)
    response_data = response.json()
    
    if 'candidates' in response_data:
        return response_data['candidates'][0]['content']['parts'][0]['text'].strip()
    else:
        return "Error: No response generated"

print("=== Demo: Related Questions ===")
print("Note: Each question is independent - no conversation history is maintained in this demo\n")

questions = [
    "What is Python?explain in few words",
    "What are the main advantages of this language?",    
]

for question in questions:
    answer = get_ai_response(question)
    print(f"Q: {question}\nA: {answer}\n")
    print("-" * 80 + "\n")

In [ ]:
# Demo: Conversation with History
# This example shows how to maintain conversation context across multiple requests

import requests
import os
from dotenv import load_dotenv

load_dotenv(override=True)

gemini_api_key = os.getenv("GEMINI_API_KEY")
if gemini_api_key is None:
    raise ValueError("GEMINI_API_KEY not found in .env file")

url = "https://generativelanguage.googleapis.com/v1beta/models/gemini-3-flash-preview:generateContent"

headers = {
    "Content-Type": "application/json",
    "x-goog-api-key": gemini_api_key
}

# Maintain conversation history in Gemini format
conversation_contents = []

def get_ai_response_with_history(user_message, max_tokens=1500, temperature=0.7):
    """Function to get AI response while maintaining conversation history"""
    # Add user message to history
    conversation_contents.append({
        "role": "user",
        "parts": [{"text": user_message}]
    })
    
    payload = {
        "contents": conversation_contents,  # Send entire conversation history
        "generationConfig": {
            "maxOutputTokens": max_tokens,
            "temperature": temperature
        }
    }
    
    response = requests.post(url, json=payload, headers=headers)
    response_data = response.json()
    
    # Extract and store AI response
    if 'candidates' in response_data:
        ai_reply = response_data['candidates'][0]['content']['parts'][0]['text'].strip()
        
        # Add assistant response to history
        conversation_contents.append({
            "role": "model",  # Gemini uses "model" instead of "assistant"
            "parts": [{"text": ai_reply}]
        })
        
        return ai_reply
    else:
        return "Error: No response generated"

print("=== Demo: Conversation with History ===")
print("Notice how the AI remembers context from previous messages!\n")
print("="*80)

# First question
first_question = "What is Python? Explain in a few words."
print(f"\n👤 User: {first_question}")
response1 = get_ai_response_with_history(first_question)
print(f"🤖 AI: {response1}")
print("-" * 80)

# Follow-up question - uses "it" referring to Python
follow_up = "What are the main advantages of it?"
print(f"\n👤 User: {follow_up}")
print("   ℹ️  Notice: 'it' refers to Python from previous context")
response2 = get_ai_response_with_history(follow_up)
print(f"🤖 AI: {response2}")
print("-" * 80)
'''
# Another follow-up - builds on the conversation
third_question = "Which of those advantages makes it best for beginners?"
print(f"\n👤 User: {third_question}")
print("   ℹ️  Notice: 'those advantages' refers to the previous answer")
response3 = get_ai_response_with_history(third_question)
print(f"🤖 AI: {response3}")
print("="*80)

print("\n💡 Key Points:")
print("• Conversation history is maintained in 'conversation_contents' list")
print("• Each turn has 'role' (user/model) and 'parts' with text")
print("• Full history is sent with each request")
print("• AI can reference 'it', 'those', 'that' from previous messages")  '''

### Complete Setup: Import Required Libraries

Setting up all necessary imports for advanced examples.

In [ ]:
# Import required libraries
import requests
import json
import os
from dotenv import load_dotenv
import time

# Load environment variables
load_dotenv(override=True)

### Configure Google Gemini API Credentials

Setting up API credentials and endpoints from environment variables.

In [ ]:
# Configure Gemini API credentials
gemini_api_key = os.getenv("GEMINI_API_KEY")
if gemini_api_key is None:
    raise ValueError("❌ GEMINI_API_KEY not found in .env file")

# Set up the API endpoint
base_url = "https://generativelanguage.googleapis.com/v1beta"
model_name = "gemini-3-flash-preview"  # You can change to gemini-2.5-flash, gemini-3-pro-preview, etc.
completion_endpoint = f"{base_url}/models/{model_name}:generateContent"

# Configure headers for API requests
headers = {
    "Content-Type": "application/json",
    "x-goog-api-key": gemini_api_key
}

print("✅ Gemini API configured successfully!")
print(f"🔗 Endpoint: {completion_endpoint}")
print(f"🤖 Model: {model_name}")
print(f"🔑 API Key: {gemini_api_key[:20]}..." if gemini_api_key else "❌ No API key found")

### Helper Functions: Create Payload and Send Requests

These functions standardize how we interact with the Gemini API.

In [ ]:
def create_gemini_payload(user_message, system_instruction=None, temperature=0.7, max_tokens=512, thinking_budget=0):
    """
    Create a standardized payload for Gemini API requests
    
    Args:
        user_message: The user's question or prompt
        system_instruction: Optional system instruction to guide model behavior
        temperature: Controls randomness (0.0-2.0)
        max_tokens: Maximum output tokens
        thinking_budget: Limit internal reasoning tokens (0 = disable thoughts)
    """
    payload = {
        "contents": [
            {
                "parts": [
                    {
                        "text": user_message
                    }
                ]
            }
        ],
        "generationConfig": {
            "temperature": temperature,
            "maxOutputTokens": max_tokens,
            "topK": 40,
            "topP": 0.95
        }
    }
    
    # Add system instruction if provided
    if system_instruction:
        payload["systemInstruction"] = {
            "parts": [
                {
                    "text": system_instruction
                }
            ]
        }
    
    return payload


def send_gemini_request(payload, show_full_response=False):
    """
    Send request to Gemini API and extract content from response
    """
    try:
        response = requests.post(completion_endpoint, headers=headers, json=payload)
        response.raise_for_status()
        
        response_data = response.json()
        
        if show_full_response:
            print("🔍 Full Response:")
            print(json.dumps(response_data, indent=2))
            print("\n" + "="*50 + "\n")
        
        # Extract content from response
        if 'candidates' in response_data:
            content = response_data['candidates'][0]['content']['parts'][0]['text']
            return content
        else:
            print("❌ No content found in response")
            if 'error' in response_data:
                print(f"Error: {response_data['error']}")
            return None
            
    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"❌ Failed to parse JSON response: {e}")
        return None

print("✅ Helper functions created successfully!")
print("📝 Functions available:")
print("   - create_gemini_payload(): Create standardized API payloads")
print("   - send_gemini_request(): Send requests and extract content")

In [ ]:
# Test Gemini connection with a simple request
test_prompt = "Hello! Please respond  with 'Gemini connection successful' to confirm the connection is working."

print("🔄 Testing Gemini API connection...")
payload = create_gemini_payload(test_prompt, temperature=0)
response = send_gemini_request(payload)

if response:
    print("✅ Gemini Connection Test Result:")
    print(f"📤 Sent: {test_prompt}")
    print(f"📥 Received: {response}")
else:
    print("❌ Connection test failed!")

## Example: Article Classification Without System Instruction

Let's see how the model responds to a classification task without any specific instructions.

In [ ]:
# Sample article for testing
california_drought_article = """
Severe drought likely in California

Millions of California residents are bracing for less water and dry lawns as drought threatens to leave a large swath of the region with a growing water shortage.

In a remarkable indication of drought severity, officials in Southern California have declared a first-of-its-kind action limiting outdoor water use to one day a week for nearly 8 million residents.

Much remains to be determined about how daily life will change as people adjust to a drier normal. But officials are warning the situation is dire and could lead to even more severe limits later in the year.
"""

# Basic prompt without specific instructions
basic_prompt = f"""What kind of article is this?
---
{california_drought_article}"""

print("📝 Basic Prompt (No System Instruction):")
print("="*60)
print(basic_prompt)
print("="*60)

## Send Basic Request (No System Instruction)

Getting a response without any specific behavioral guidance.

In [ ]:
# Send basic prompt and see response
payload = create_gemini_payload(basic_prompt, temperature=0)
response = send_gemini_request(payload)

print("🤖 Basic Response (No System Instruction):")
print("="*60)
if response:
    print(response)
else:
    print("❌ No response received")
print("="*60)

## Example: Article Classification WITH System Instruction

Now let's use a system instruction to guide the model's behavior for concise responses.

In [ ]:
# Configure system instruction for specific behavior
system_instruction = "You are a news aggregator that categorizes news articles. Respond with a single category word only."

# Same user prompt but with system instruction
user_prompt = f"""What kind of article is this?
---
{california_drought_article}"""

print("🎯 System Instruction:")
print("="*60)
print(system_instruction)
print("="*60)

print("\n📝 User Prompt:")
print("="*60)
print(user_prompt)
print("="*60)

## Send Request WITH System Instruction

Compare the difference when we guide the model's behavior.

In [ ]:
# Send request with system instruction
payload = create_gemini_payload(user_prompt, system_instruction=system_instruction, temperature=0)
response = send_gemini_request(payload)

print("🤖 Response WITH System Instruction:")
print("="*60)
if response:
    print(response)
else:
    print("❌ No response received")
print("="*60)

print("\n📊 Comparison:")
print("• Without system instruction: Likely gives detailed description")
print("• With system instruction: Should give single category word")
print("\n💡 System instructions are powerful for controlling output format and behavior!")